In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
cmap = matplotlib.colormaps.get('tab10').colors
import torch
import torch_geometric as tg
from gen_autoencoder_dataset import AutoencoderDataset
from torcheval.metrics import BinaryAUROC
from tqdm import trange

In [2]:
n = 30
dataset = AutoencoderDataset(root='autoencoder_graphs', n_graphs_per_type=n)
dataset.process()
train_loader = tg.loader.DataLoader(dataset[:n*12], batch_size=16, shuffle=True)
test_loader = tg.loader.DataLoader(dataset[n*12:], batch_size=16, shuffle=True)
train_loader.dataset[2]
device= 'cuda' if torch.cuda.is_available() else 'cpu'
data = dataset[0]

100%|██████████| 420/420 [00:10<00:00, 40.03it/s]
c:\Users\steph\VS Code\Python\Computerpraktikum\gen_autoencoder_dataset.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


## New idea: Do on my own

In [3]:
num_features = dataset.num_features
alpha = 0.5 # weight of the attribute loss


class dominant(torch.nn.Module):
    def __init__(self):
        super(dominant, self).__init__()
        self.encoder = torch.nn.ModuleList()
        self.encoder.append(tg.nn.GCNConv(num_features, 3))
        self.encoder.append(tg.nn.GCNConv(3, 3))
        self.encoder.append(tg.nn.GCNConv(3, 2))

        self.attr_decoder =torch.nn.ModuleList()
        self.attr_decoder.append(tg.nn.GCNConv(2, 3))
        self.attr_decoder.append(tg.nn.GCNConv(3, 3))
        self.attr_decoder.append(tg.nn.GCNConv(3, num_features))

        #self.dropout = torch.nn.Dropout(0.2)  
    def forward(self, x, edge_index):
        z = self.encoder[0](x, edge_index).relu()
        for layer in self.encoder[1:]:
            z = layer(z, edge_index).relu()
        
        # decoder
        adj_ = torch.sigmoid(z @ z.T) # structure decoder
        x_ = self.attr_decoder[0](z, edge_index).relu()
        for layer in self.attr_decoder[1:]:
            x_ = layer(x_, edge_index).relu()
        return x_, adj_



model = dominant()
model.to(device)
print(model)

def loss_func(x, x_, adj, adj_):
    adj = adj.type(torch.float)
    return alpha * torch.nn.functional.mse_loss(x, x_) + (1-alpha) * torch.nn.functional.mse_loss(adj, adj_)

optimizer = torch.optim.Adam(model.parameters())

dominant(
  (encoder): ModuleList(
    (0): GCNConv(4, 3)
    (1): GCNConv(3, 3)
    (2): GCNConv(3, 2)
  )
  (attr_decoder): ModuleList(
    (0): GCNConv(2, 3)
    (1): GCNConv(3, 3)
    (2): GCNConv(3, 4)
  )
)


In [4]:
# training:
epochs = 10
train_loss= []
def train(loader):
    model.train()
    avg_loss = 0
    for data in loader:
        data = data.to(device)
        x_, adj = model.forward(data.x, data.edge_index)

        loss = loss_func(data.x, x_, tg.utils.to_dense_adj(data.edge_index, max_num_nodes=len(x_))[0], adj)
        avg_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return avg_loss / len(loader)
def test(loader):
    model.eval()
    avg_loss = 0
    for data in loader:
        data = data.to(device)
        x_, adj = model.forward(data.x, data.edge_index)
        loss = loss_func(data.x, x_, tg.utils.to_dense_adj(data.edge_index, max_num_nodes=len(x_))[0], adj)
        avg_loss += loss.item()
    return avg_loss / len(loader)

print('Initial Test Loss: {:.3f}'.format(test(test_loader)))
for epoch in range(1, epochs+1):
    loss = train(train_loader)
    train_loss.append(loss)
    print(f'Epoch: {epoch:03d}')
    print('\tTraining Loss: {:.3f}'.format(loss))
    print('\tTest Loss: {:.3f}'.format(test(test_loader)))

Initial Test Loss: 0.187
Epoch: 001
	Training Loss: 0.187
	Test Loss: 0.183
Epoch: 002
	Training Loss: 0.183
	Test Loss: 0.177
Epoch: 003
	Training Loss: 0.179
	Test Loss: 0.173
Epoch: 004
	Training Loss: 0.174
	Test Loss: 0.170
Epoch: 005
	Training Loss: 0.171
	Test Loss: 0.169
Epoch: 006
	Training Loss: 0.168
	Test Loss: 0.167
Epoch: 007
	Training Loss: 0.167
	Test Loss: 0.164
Epoch: 008
	Training Loss: 0.165
	Test Loss: 0.164
Epoch: 009
	Training Loss: 0.165
	Test Loss: 0.162
Epoch: 010
	Training Loss: 0.164
	Test Loss: 0.162


In [26]:
metric = BinaryAUROC()
model.eval()
data = dataset[-9]
x, adj = model.forward(data.x, data.edge_index)
metric.update(adj.flatten(), tg.utils.to_dense_adj(data.edge_index, max_num_nodes=len(data.x))[0].flatten())
metric.compute()
# import sklearn
# #sklearn.preprocessing.minmax_scale((adj - tg.utils.to_dense_adj(data.edge_index, max_num_nodes=len(data.x))[0]).detach().numpy()**2)
# adjtrue = tg.utils.to_dense_adj(data.edge_index, max_num_nodes=len(data.x))[0].flatten()
# sklearn.metrics.average_precision_score(adj.detach().numpy().flatten(), adjtrue)

c:\Users\steph\VS Code\Python\Computerpraktikum\gen_autoencoder_dataset.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(os.path.join(self.processed_di

tensor(0.6460, dtype=torch.float64)